In [11]:
# basic operations
import numpy as np
import pandas as pd

# Plotly
import plotly.express as px
import plotly.graph_objects as go

# SQL
%run -i "Query_Function.py"

In [89]:
# helper functions

def rev_stat(tag_col, f, df):
    mask = tag_col>0
    rev = df.loc[mask,'Royalties']
    return f(rev)

def generate_df_tags(df):
    # Generate tag stats
    stats = {}
    stats['Count'] = df.iloc[:,5:].apply(rev_stat, args=[len, df])
    stats['Median'] = df.iloc[:,5:].apply(rev_stat, args=[np.median, df])
    stats['Mean'] = df.iloc[:,5:].apply(rev_stat, args=[np.mean, df])
    stats['Sum'] = df.iloc[:,5:].apply(rev_stat, args=[np.sum, df])
    stats['STD'] = df.iloc[:,5:].apply(rev_stat, args=[np.std, df])

    # Save to a df
    df_tags = pd.DataFrame(stats)
    df_tags.index.name = 'Tag'
    df_tags = df_tags.reset_index()
    
    return df_tags

def query_tag(df, tag):
    mask = df[tag]>0
    return df[mask]

# Generate a dataframe for analysis
def pipeline(df, country_in, country_out, horizen):
    
    # horizen = -1 means taking the whole horizen
    if horizen == -1:
        horizen = pd.Timedelta(99999, unit='M')
    else:
        horizen = pd.Timedelta(horizen, unit='M') 
    
    # Filter by country
    mask = df['Country_of_origin'].isin(country_in) & df['Country_of_market'].isin(country_out)
    df = df[mask]
    
    # Filter by horizen
    df_temp = df.groupby(['Mama_id','Movie_id','Title','Country_of_origin', 'Country_of_market'])
    mask = df_temp['Date'].apply(lambda x:(x - min(x))<=horizen)
    df = df[mask]
    
    # Aggregate by country
    df = df.groupby(['Mama_id','Movie_id','Title','Country_of_origin', 'Country_of_market']).sum()
    df = df.reset_index().set_index('Mama_id')

    # Merge with tags
    df = df.merge(tags,left_index=True, right_index=True, how='left')
   
    # Generate a df for EDA
    df_tags = generate_df_tags(df)
    return df, df_tags

In [13]:
# Load revenue data by day
query = '''
SELECT s.Movie_id AS Movie_id, 
       m.Mama_id AS Mama_id,
       m.Title AS Title,
       m.Nationality AS Country_of_origin,
       s.Country AS Country_of_market,
       s.Royalties AS Royalties,
       s.Date AS Date

FROM (SELECT SUM(s.royalties_paid) AS Royalties,
             s.movie_metadata_id AS Movie_id,
             u.country_name AS Country,
             d.date_value AS Date

     FROM fact_sales AS s
     INNER JOIN dim_user as u
     ON s.user_id = u.user_id
     INNER JOIN dim_date AS d
     ON s.download_date_pst_id = d.date_id

     GROUP BY 2,3,4) AS s
INNER JOIN (SELECT movie_metadata_id AS Movie_id,
                   mama_movie_id AS Mama_id,
                   original_title AS Title,
                   nationality AS Nationality
            FROM umw_dwh.dim_movie_metadata) AS m
ON s.Movie_id = m.Movie_id
'''

#df = data(query)

In [14]:
# Save queried data
#df.to_csv('movie_revenue_new_by_day.csv', index=None)

# Load Local Data

In [81]:
# Read tags and movie revenue data
tags = pd.read_csv("tags_clean.csv", index_col='id')
df = pd.read_csv('movie_revenue_new_by_day.csv', parse_dates=['Date'])

# Variables

In [6]:
# Country-in
df['Country_of_origin'].unique()

array([nan, 'US', 'BE', 'FR', 'ES', 'CA', 'DE', 'SG', 'IN', 'MY', 'NL',
       'HK', 'GB', 'AT', 'LT', 'JP', 'RU', 'ZA', 'LU', 'CN', 'DK', 'AR',
       'KR', 'TH', 'IS', 'VN', 'IL', 'NZ', 'AU', 'HR', 'IT', 'MX', 'SN',
       'MA', 'IE', 'PL', 'SI', 'RO', 'CZ', 'NO', 'TR', 'CO', 'CH', 'SE',
       'NP', 'BR', 'UY', 'PH', 'FI', 'PE', 'TW', 'PT', 'CL', 'KZ', 'SA',
       'TN', 'ID', 'BG', 'IR', 'CU', 'AF', 'HU', 'LB', 'JO', 'EG', 'RS',
       'NG', 'MK', 'AM', 'EE', 'PR', 'GT', 'UM', 'GR', 'LV', 'TT', 'VE',
       'GE', 'EC', 'CD', 'ET', 'AE', 'UA', 'BA', 'CY', 'SK', 'MC', 'AO',
       'NE', 'DO', 'ZM', 'PY', 'ER'], dtype=object)

In [7]:
# Country-out
df['Country_of_market'].unique()

array(['Australia', 'Austria', 'Belgium', 'Brazil', 'Canada', 'Denmark',
       'Finland', 'France', 'Germany', 'Greece', 'Hong Kong', 'Indonesia',
       'Ireland', 'Italy', 'Japan', 'Luxembourg', 'Malaysia', 'Mexico',
       'Netherlands', 'New Zealand', 'Norway', 'Poland',
       'Russian Federation', 'Singapore', 'South Africa', 'Spain',
       'Sweden', 'Switzerland', 'Taiwan, Province of China',
       'United Kingdom', 'United States', 'Argentina',
       'Brunei Darussalam', 'Chile', 'Colombia', 'Costa Rica',
       'Dominican Republic', 'Ecuador', 'Guatemala', 'Macao', 'Panama',
       'Paraguay', 'Peru', 'Philippines', 'Sri Lanka', 'Thailand',
       'Viet Nam', 'El Salvador', 'Hungary', 'Israel', 'Slovakia',
       'Cambodia', 'Honduras', 'Venezuela, Bolivarian Republic of',
       'Portugal', 'Nicaragua', 'Belarus', 'Bulgaria', 'Mozambique',
       'Bolivia, Plurinational State of', 'Bermuda', 'Cyprus', 'Estonia',
       'Slovenia', 'Czech Republic', 'Lithuania', 'Malta', '

In [72]:
# Set varaiables
country_in = ['FR']
country_out = ['United States']
horizen = 3 # months

In [90]:
# Create Dataframes for use
df_query, df_tags = pipeline(df, country_in, country_out, horizen)

Query_Function.py:35: FutureWarning:

M and Y units are deprecated and will be removed in a future version.



In [86]:
# Plot the the top 20 tags with at least 10 movies
data = df_tags[df_tags['Count']>10].sort_values('Median',ascending=False)
data = data.iloc[:20]

fig = px.bar(data, x='Tag', y='Median')
fig.update_layout(title_text='Top 20 Tags with at least 10 movies')
fig.show()

In [93]:
# Query a tag
query_tag(df_query, 'false identities')

,Movie_id,Title,Country_of_origin,Country_of_market,Royalties,installation art,superheroes,Irish conflicts,1990s,spies,...,frenemies,hot pursuits,live concerts,in the skies,zombies,climbing,American Revolutionary War,looking for a parent,bars and restaurants,Western United States
Mama_id,,,,,,,,,,,,,,,,,,,,,
269,259,Astérix le Gaulois,FR,United States,826.50,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
546,520,Le marquis,FR,United States,227.50,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
579,552,Switch,FR,United States,2796.00,0.0,0.0,0.0,0.0,0.0,...,0.0,85.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
700,636,Fantômas,FR,United States,853.50,0.0,0.0,0.0,0.0,0.0,...,100.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
711,646,Les patriotes,FR,United States,153.00,0.0,0.0,0.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
868,775,Gardiens de l'ordre,FR,United States,1130.50,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
948,842,Parlez-moi de vous,FR,United States,2021.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,886,L'oncle Charles,FR,United States,1392.50,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1353,1230,Mes héros,FR,United States,1239.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
